In [21]:
import pickle
from tqdm.autonotebook import tqdm
import nltk
from typing import Dict, Any, Union
from nltk.stem import WordNetLemmatizer
from nltk.stem import LancasterStemmer
from nltk.tokenize import word_tokenize, regexp_tokenize
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /Users/far/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/far/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/far/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
wnl = WordNetLemmatizer()
lancaster = LancasterStemmer()

In [23]:
HRF = pd.read_csv('Hotel_Reviews.csv')
HRF.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [43]:
stop_words_list = stopwords.words('english')

neg_words_by_nation_dict: Dict[str, Dict[str, Union[int, Any]]] = {}
pos_words_by_nation_dict: Dict[str, Dict[str, Union[int, Any]]] = {}


def process_review_word(word):
    if word.isdecimal():
        return None
    word_lower = word.lower()
    if word_lower in stop_words_list:
        return None

    word_lemmatized = wnl.lemmatize(word_lower, pos='v')
#     word_stem = lancaster.stem(word_lemmatized)
    word = word_lemmatized

    return word


def split_words(words):
    #     return neg_review.split()
    return regexp_tokenize(words, pattern="\w+")


for index, row in tqdm(HRF.iterrows(), total=HRF.shape[0]):
    neg_review = row["Negative_Review"]
    pos_review = row["Positive_Review"]
    reviewer_nation = row["Reviewer_Nationality"]
    reviewer_nation = reviewer_nation.strip()

    # Process negative review
    if neg_review != 'No Negative':
        neg_words = split_words(neg_review)
        for word in neg_words:
            cleaned_word = process_review_word(word)
            if cleaned_word == None:
                continue

            nation_dict = neg_words_by_nation_dict.get(reviewer_nation, {})
            nation_dict[cleaned_word] = nation_dict.get(cleaned_word, 0) + 1
            neg_words_by_nation_dict[reviewer_nation] = nation_dict

    # Process negative review
    if pos_review != 'No Positive':
        pos_words = split_words(pos_review)
        for word in pos_words:
            cleaned_word = process_review_word(word)
            if cleaned_word == None:
                continue

            nation_dict = pos_words_by_nation_dict.get(reviewer_nation, {})
            nation_dict[cleaned_word] = nation_dict.get(cleaned_word, 0) + 1
            pos_words_by_nation_dict[reviewer_nation] = nation_dict

print(len(pos_words_by_nation_dict.items()))
print(len(neg_words_by_nation_dict.items()))

{'Russia': {'park': 191,
  'outside': 22,
  'hotel': 1168,
  'beautiful': 115,
  'nice': 866,
  'locate': 114,
  'ca': 2,
  'minutes': 230,
  'walk': 356,
  'city': 265,
  'center': 251,
  'typical': 5,
  'amsterdam': 52,
  'since': 7,
  'hotels': 47,
  'small': 143,
  'build': 71,
  'pay': 28,
  'attention': 15,
  'fact': 7,
  'room': 1626,
  'two': 49,
  'storeyed': 1,
  'use': 31,
  'stairs': 5,
  'get': 136,
  'bed': 347,
  'location': 1750,
  'good': 1436,
  'take': 57,
  'mins': 46,
  'feet': 4,
  'tram': 76,
  'property': 15,
  'stylish': 70,
  'friendly': 616,
  'staff': 1027,
  'spacious': 135,
  'except': 12,
  'one': 123,
  'close': 324,
  'couple': 21,
  'stop': 64,
  'standard': 23,
  'amaze': 144,
  'adorable': 1,
  'everything': 320,
  'stun': 5,
  'z': 1,
  'indoors': 1,
  'design': 220,
  'really': 259,
  'noise': 28,
  'isolation': 7,
  'great': 693,
  'food': 105,
  'bar': 114,
  'service': 204,
  'arrive': 22,
  'tell': 5,
  'problems': 21,
  'boiler': 1,
  'would':

In [69]:
dict(list(pos_words_by_nation_dict.items())[0:1])

{'Russia': {'park': 191,
  'outside': 22,
  'hotel': 1168,
  'beautiful': 115,
  'nice': 866,
  'locate': 114,
  'ca': 2,
  'minutes': 230,
  'walk': 356,
  'city': 265,
  'center': 251,
  'typical': 5,
  'amsterdam': 52,
  'since': 7,
  'hotels': 47,
  'small': 143,
  'build': 71,
  'pay': 28,
  'attention': 15,
  'fact': 7,
  'room': 1626,
  'two': 49,
  'storeyed': 1,
  'use': 31,
  'stairs': 5,
  'get': 136,
  'bed': 347,
  'location': 1750,
  'good': 1436,
  'take': 57,
  'mins': 46,
  'feet': 4,
  'tram': 76,
  'property': 15,
  'stylish': 70,
  'friendly': 616,
  'staff': 1027,
  'spacious': 135,
  'except': 12,
  'one': 123,
  'close': 324,
  'couple': 21,
  'stop': 64,
  'standard': 23,
  'amaze': 144,
  'adorable': 1,
  'everything': 320,
  'stun': 5,
  'z': 1,
  'indoors': 1,
  'design': 220,
  'really': 259,
  'noise': 28,
  'isolation': 7,
  'great': 693,
  'food': 105,
  'bar': 114,
  'service': 204,
  'arrive': 22,
  'tell': 5,
  'problems': 21,
  'boiler': 1,
  'would':

In [70]:
dict(list(neg_words_by_nation_dict.items())[0:1])

{'Russia': {'angry': 1,
  'make': 148,
  'post': 4,
  'available': 38,
  'via': 18,
  'possible': 34,
  'sit': 16,
  'use': 120,
  'plan': 13,
  'trip': 20,
  'one': 304,
  'mistake': 14,
  'book': 203,
  'place': 129,
  'com': 43,
  'stay': 277,
  'nights': 43,
  'hotel': 923,
  'july': 5,
  'upon': 15,
  'arrival': 34,
  'small': 411,
  'room': 1938,
  '2nd': 7,
  'floor': 225,
  'turn': 40,
  'specially': 4,
  'reserve': 14,
  'level': 36,
  'duplex': 4,
  'would': 168,
  'big': 61,
  'windows': 101,
  'high': 56,
  'ceilings': 4,
  'ok': 71,
  'mind': 12,
  'break': 67,
  'window': 126,
  'close': 86,
  'hello': 3,
  'rain': 4,
  'mini': 27,
  'fridge': 24,
  'contain': 3,
  'sort': 9,
  'bio': 1,
  'weapon': 1,
  'least': 29,
  'guess': 26,
  'smell': 79,
  'intimately': 1,
  'ask': 164,
  'change': 103,
  'explain': 8,
  'time': 244,
  'btw': 1,
  'cost': 45,
  'simple': 16,
  'double': 71,
  'get': 232,
  'way': 48,
  'volume': 3,
  'due': 26,
  'ceiling': 17,
  'offer': 57,
  '

In [67]:
# Save word count cache
with open('pos_words_by_nation.pickle', 'wb') as pickle_file:
    pickle_file.write(pickle.dumps(pos_words_by_nation_dict))

with open('neg_words_by_nation.pickle', 'wb') as pickle_file:
    pickle_file.write(pickle.dumps(neg_words_by_nation_dict))

In [68]:
# Load cached word count
with open('pos_words_by_nation.pickle', 'rb') as pickle_file:
    pos_words_by_nation_dict = pickle.load(pickle_file)

with open('neg_words_by_nation.pickle', 'rb') as pickle_file:
    neg_words_by_nation_dict = pickle.load(pickle_file)

print(len(pos_words_by_nation_dict.items()))
print(len(neg_words_by_nation_dict.items()))

224
222


In [71]:
arrange = list()
min_max_neg_words_count_by_nation_dict = dict()
min_max_pos_words_count_by_nation_dict = dict()


def process_words_by_nation_dict(words_dict, positive):
    for nation, nation_dict in tqdm(words_dict.items()):
        word_arrange = list()

        for word, count in nation_dict.items():
            newt = (count, word)
            word_arrange.append(newt)

        word_arrange = sorted(word_arrange, reverse=True)
        word_arrange = word_arrange[:100]
        min_count = None
        max_count = None

        for word_pair in word_arrange:
            count, word = word_pair
            newt = (nation, count, positive, word)
            arrange.append(newt)

            if max_count is None or max_count < count:
                max_count = count
            if min_count is None or min_count > count:
                min_count = count

        # Save min/max to dict
        if positive is True:
            min_max_pos_words_count_by_nation_dict[nation] = (
                min_count, max_count)
        else:
            min_max_neg_words_count_by_nation_dict[nation] = (
                min_count, max_count)


process_words_by_nation_dict(neg_words_by_nation_dict, False)
process_words_by_nation_dict(pos_words_by_nation_dict, True)

dict(list(min_max_pos_words_count_by_nation_dict.items())[0:10])

{'Russia': (72, 1750),
 'Ireland': (240, 6584),
 'Australia': (423, 9714),
 'United Kingdom': (3761, 99403),
 'New Zealand': (58, 1427),
 'Poland': (47, 1076),
 'Norway': (38, 875),
 'France': (110, 2380),
 'Italy': (90, 2113),
 'Canada': (161, 3492)}

In [72]:
# Spread the font sizes across 20-100 based on the count
big_size = 160
small_size = 5

words_by_nation_csv_file = open('words_by_nation.csv', 'w')
words_by_nation_csv_file.write("country,text,positive,count,size\n")

for item in tqdm(arrange):
    nation, count, positive, word = item

    if positive is True:
        min_count, max_count = min_max_pos_words_count_by_nation_dict.get(
            nation)
    else:
        min_count, max_count = min_max_neg_words_count_by_nation_dict.get(
            nation)

    if min_count == max_count:
        size = 1
    else:
        size = (count - min_count) / float(max_count - min_count)
    size = int((size * big_size) + small_size)

    words_by_nation_csv_file.write(
        '{},{},{},{},{}\n'.format(nation, word, positive, str(count), str(size)))
words_by_nation_csv_file.close()

!head words_by_nation.csv


country,text,postive,count,size
Russia,room,False,1938,165
Russia,hotel,False,923,77
Russia,breakfast,False,558,46
Russia,small,False,411,33
Russia,one,False,304,24
Russia,stay,False,277,22
Russia,work,False,259,20
Russia,bite,False,251,19
Russia,bed,False,249,19
